# QLoRA Fine-tune Mistral 7B (Colab/Kaggle)

Steps: install deps, mount Drive (optional), load our chat shards from `data/sft/chat_mistral/*.jsonl`, run SFTTrainer with LoRA, then save adapters.


In [ ]:
!pip -q install transformers peft datasets accelerate bitsandbytes trl


In [ ]:
import glob, json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
import torch

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
shards = glob.glob('data/sft/chat_mistral/*.jsonl')
assert shards, 'No shards found; upload or generate via scripts/build_sft_dataset.py'

def load_chat_dataset(paths):
    records = []
    for p in paths:
        with open(p, 'r', encoding='utf-8') as f:
            for line in f:
                if not line.strip():
                    continue
                records.append(json.loads(line))
    return Dataset.from_list(records)

ds = load_chat_dataset(shards)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, load_in_4bit=True, device_map='auto', torch_dtype=torch.float16
)

peft_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_dropout=0.05, bias='none', task_type='CAUSAL_LM'
)
model = get_peft_model(model, peft_config)

args = TrainingArguments(
    output_dir='qlora-mistral', per_device_train_batch_size=1, gradient_accumulation_steps=8,
    learning_rate=2e-4, fp16=True, logging_steps=10, num_train_epochs=1, save_strategy='no', report_to='none'
)

trainer = SFTTrainer(
    model=model, tokenizer=tokenizer, train_dataset=ds, max_seq_length=2048,
    args=args, dataset_text_field='text'
)
trainer.train()
model.save_pretrained('qlora-mistral')
tokenizer.save_pretrained('qlora-mistral')
